# Get list of users from Active Directory and write to csv

In [5]:
import csv
from pyad import adsearch
from datetime import datetime
from os import listdir

### Functions

In [6]:
import csv
from pyad import adsearch

def extrFirstLastName(displayName):
        firstName, lastName = '', ''
    
        for namePart in displayName.split(' '):
            if namePart == namePart.upper():
                lastName += namePart+' '
            else:
                firstName += namePart+' '
                
        firstName, lastName = firstName.strip(), lastName.strip()
        
        return [firstName, lastName]
    

def extrManagerName(manager):
    for item in manager.split(','):
        if item[:3] == 'CN=':
            manager = item[3:]
            return manager
    return ''


def newUsers(augUsers, lastUsers):
    newUserL = []
    for currentUser in augUsers:
        if currentUser['displayName'] not in [item['displayName'] for item in lastUsers]:
            newUserL.append({'name': currentUser['displayName']})
            
    if newUserL:
        fileName = 'new_augUsers' + datetime.now().strftime("%y%m%d_%H%M%S") + '.csv'
        writeCSV(fileName, newUserL)
    
    return


def leftUsers(augUsers, lastUsers):
    leftUserL = []
    for oldUser in lastUsers:
        if oldUser['displayName'] not in [item['displayName'] for item in augUsers]:
            leftUserL.append({'name':oldUser['displayName']})
            
    if leftUserL:
        fileName = 'left_augUsers' + datetime.now().strftime("%y%m%d_%H%M%S") + '.csv'
        writeCSV(fileName, leftUserL)
            
    return


def newManagers(augUsers, lastUsers):
    newManagerL = []
    for currentUser in augUsers:
        currentManager = currentUser['manager']
        oldManagerL = [item['manager'] for item in lastUsers if currentUser['displayName'] == item['displayName']]

        if oldManagerL: 
            oldManager = oldManagerL[0]
        else:
            oldManager = ''

        if currentManager != oldManager and currentManager and oldManager:
            newManagerL.append({'name':currentUser['displayName'], 
                                'oldManager':extrManagerName(oldManager), 
                                'newManager':extrManagerName(currentManager) })
    if newManagerL:
        fileName = 'newMngr_augUsers' + datetime.now().strftime("%y%m%d_%H%M%S") + '.csv'
        writeCSV(fileName, newManagerL)
    
    return

# - - - - - -

def writeCSV(fileName, listOfDict):
    attributeList = list(listOfDict[-1].keys())
    csv.register_dialect('myDialect', delimiter = ',' , lineterminator = '\n', quoting=csv.QUOTE_ALL)

    print('... writing {}'.format(fileName))

    with open(fileName, mode='w') as csv_file:
        writer = csv.DictWriter(csv_file, fieldnames=attributeList, dialect='myDialect')
        writer.writeheader()
        writer.writerows(listOfDict)   

    csv_file.close()
    return

### get users from AD

In [7]:
print('= = = Get Users From AD = = =')
print('... getting users ...')

q = adsearch.ADQuery()
attributeList = ['displayName', 'mail', 'title', 'telephoneNumber', 'mobile', 'manager', 'directReports', 'physicalDeliveryOfficeName']
q.execute_query( attributes=attributeList, where_clause = "objectClass = '*'", base_dn = "OU=Users,OU=AUG,OU=DE,DC=ls,DC=ege,DC=ds")

augUsers = [ row for row in q.get_results() if row['displayName']  ]

= = = Get Users From AD = = =
... getting users ...


In [15]:
print(augUsers)

[{'physicalDeliveryOfficeName': 'AUGSBURG R&D', 'directReports': None, 'manager': 'CN=NEAV Yongchin,OU=Users,OU=BRO,OU=FR,DC=ls,DC=ege,DC=ds', 'mobile': '+491774724315', 'telephoneNumber': '+491774724315', 'title': 'Program Manufacturing leader', 'mail': 'denis.thomas@faurecia.com', 'displayName': 'THOMAS Denis', 'firstName': 'Denis', 'lastName': 'THOMAS', 'managerName': 'NEAV Yongchin'}, {'physicalDeliveryOfficeName': 'AUGSBURG R&D', 'directReports': None, 'manager': 'CN=BERMEITINGER Roland,OU=Users,OU=AUG,OU=DE,DC=ls,DC=ege,DC=ds', 'mobile': None, 'telephoneNumber': '+498214103372', 'title': 'Technischer Service Elektrotechnik', 'mail': 'raphael.wittkopf@faurecia.com', 'displayName': 'WITTKOPF Raphael', 'firstName': 'Raphael', 'lastName': 'WITTKOPF', 'managerName': 'BERMEITINGER Roland'}, {'physicalDeliveryOfficeName': 'AUGSBURG R&D', 'directReports': ('CN=FOYARD Danae (DFD) (external),OU=Users,OU=BAV,OU=FR,DC=ls,DC=ege,DC=ds', 'CN=POCINHO Nicolas (external),OU=Users,OU=BAV,OU=FR,DC=

import csv
keys = augUsers[0].keys()
with open(r'D:\users3.csv', 'w') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(augUsers)

### name processing

In [37]:
for i,user in enumerate(augUsers):
    augUsers[i]['firstName'], augUsers[i]['lastName'] = extrFirstLastName(user['displayName'])
    
    if user['manager']:
        managerName = extrManagerName(user['manager'])
        augUsers[i]['managerName']= managerName

### read recent users from csv

In [59]:
li = [1,2,12,25,23,22,3,3,5,4,8]
print(li[:8])

[1, 2, 12, 25, 23, 22, 3, 3]


In [72]:
print('... comparing to recent users')

#lastfilesL = [item for item in listdir('./') if 'augUsers' == item[:8]]
lastfilesL = [item for item in listdir('D:/') if 'augUsers' == item[:8]]
#print(lastfilesL)
if lastfilesL:
    lastfile = sorted(lastfilesL)[-1]
    #print(type(lastfile),"hi")
    print('... comparing to file: {}'.format(lastfile))

    with open('D:/'+lastfile) as f:
         lastUsers = [{k: v for k, v in row.items()}
              for row in csv.DictReader(f, skipinitialspace=True)]

... comparing to recent users
<class 'str'> hi
... comparing to file: augUsers.csv


In [73]:
print(lastUsers)

[{'physicalDeliveryOfficeName': 'AUGSBURG R&D', 'directReports': '', 'manager': 'CN=NEAV Yongchin,OU=Users,OU=BRO,OU=FR,DC=ls,DC=ege,DC=ds', 'mobile': '+491774724315', 'telephoneNumber': '+491774724315', 'title': 'Program Manufacturing leader', 'mail': 'denis.thomas@faurecia.com', 'displayName': 'THOMAS Denis', 'firstName': 'Denis', 'lastName': 'THOMAS', 'managerName': 'NEAV Yongchin'}, {'physicalDeliveryOfficeName': 'AUGSBURG R&D', 'directReports': '', 'manager': 'CN=BERMEITINGER Roland,OU=Users,OU=AUG,OU=DE,DC=ls,DC=ege,DC=ds', 'mobile': '', 'telephoneNumber': '+498214103372', 'title': 'Technischer Service Elektrotechnik', 'mail': 'raphael.wittkopf@faurecia.com', 'displayName': 'WITTKOPF Raphael', 'firstName': 'Raphael', 'lastName': 'WITTKOPF', 'managerName': 'BERMEITINGER Roland'}, {'physicalDeliveryOfficeName': 'AUGSBURG R&D', 'directReports': "('CN=FOYARD Danae (DFD) (external),OU=Users,OU=BAV,OU=FR,DC=ls,DC=ege,DC=ds', 'CN=POCINHO Nicolas (external),OU=Users,OU=BAV,OU=FR,DC=ls,DC

### users with new manager

In [74]:
newManagerL = []
for currentUser in augUsers:
    currentManager = currentUser['manager']
    oldManagerL = [item['manager'] for item in lastUsers if currentUser['displayName'] == item['displayName']]

    if oldManagerL: 
        oldManager = oldManagerL[0]
    else:
        oldManager = ''

    if currentManager != oldManager and currentManager and oldManager:
        newManagerL.append({'name':currentUser['displayName'], 
                            'oldManager':extrManagerName(oldManager), 
                            'newManager':extrManagerName(currentManager) })

In [82]:
print(oldManagerL[0])

CN=BREM Josef,OU=Users,OU=AUG,OU=DE,DC=ls,DC=ege,DC=ds


### users left

In [75]:
leftUserL = []
for oldUser in lastUsers:
    if oldUser['displayName'] not in [item['displayName'] for item in augUsers]:
        leftUserL.append({'name':oldUser['displayName']})

In [79]:
print(leftUserL)

[]


### new users

In [76]:
newUserL = []
for currentUser in augUsers:
    if currentUser['displayName'] not in [item['displayName'] for item in lastUsers]:
        newUserL.append({'name': currentUser['displayName']})

In [78]:
print(newUserL)

[]
